In [1]:
import os
import cleaning
import numpy as np

In [2]:
import malaya

In [4]:
from glob import glob

files = glob('../pure-text/dumping-*.txt')
files

['../pure-text/dumping-instagram.txt',
 '../pure-text/dumping-parliament.txt',
 '../pure-text/dumping-twitter.txt',
 '../pure-text/dumping-iium.txt',
 '../pure-text/dumping-wiki.txt',
 '../pure-text/dumping-news.txt',
 '../pure-text/dumping-watpadd.txt',
 '../pure-text/dumping-pdf.txt',
 '../pure-text/dumping-common-crawl.txt']

In [8]:
glob('../pure-text/*.json')

['../pure-text/iium.json',
 '../pure-text/2020-02-22-twitter-dump-in.json',
 '../pure-text/manglish.json',
 '../pure-text/dumping-twitter-6-july-2019.json',
 '../pure-text/2020-03-08-twitter-dump-in.json',
 '../pure-text/academia-pdf.json',
 '../pure-text/dumping-instagram-6-july-2019.json']

In [12]:
with open('../pure-text/dumping-pdf.txt') as fopen:
    data = fopen.read().split('\n')
    
data[:10]

['JABATAN USULUDDIN',
 'DAN FALSAFAH',
 '',
 'KOLOKIUM SISWAZAH',
 'JABATAN USULUDDIN DAN FALSAFAH',
 '',
 'Mazlan Ibrahim',
 'Nurulfathonah Mohd Effendy',
 'Nur Fathiah A Rozak',
 'Muhammad Akmal R . Azmi']

In [15]:
results, result = [], []
for i in data:
    if not len(i) and len(result):
        results.append(' '.join(result))
        result = []
    else:
        result.append(i)

In [33]:
selected = []
for r in tqdm(results):
    if r[:30].isupper():
        continue
    if len(r) < 200:
        continue
    if 'ISBN' in r or 'cipta terpelihara' in r or 'rights reserved' in r:
        continue
    selected.extend(malaya.text.function.split_into_sentences(r, minimum_length = 30))

In [37]:
import tensorflow_text  
import tensorflow as tf

tf.compat.v1.reset_default_graph()
sess = tf.InteractiveSession()
meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], '../t5/output/1598704247')
signature_def = meta_graph_def.signature_def["serving_default"]
pred = lambda x: sess.run(
    fetches=signature_def.outputs["outputs"].name, 
    feed_dict={signature_def.inputs["input"].name: x}
)

INFO:tensorflow:Restoring parameters from ../t5/output/1598704247/variables/variables


In [50]:
len(selected) // 20 * 20

258200

In [51]:
from tqdm import tqdm
import re

def cleaning(string):
    string = re.sub('[^\'"A-Za-z\- ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string.lower()).strip()
    return string

batch_size = 20
output = []

for i in tqdm(range(0, len(selected) // batch_size * batch_size, batch_size)):
    batch = selected[i: i + batch_size]
    batch_x = [f'parafrasa: {s}' for s in batch]
    r = pred(batch_x)
    
    for k in range(batch_size):
        if cleaning(batch[k]) == cleaning(r[k].decode()):
            continue
        output.append((batch[k], r[k].decode()))

100%|██████████| 12910/12910 [16:07:21<00:00,  4.50s/it]   


In [52]:
len(output)

246667

In [54]:
import json

with open('semisupervised-paraphrase-pdf.json', 'w') as fopen:
    json.dump(output, fopen)